In [1]:
import torch
import torch.optim as optim
import pandas as pd
import numpy as np

In [1]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available


False


/home/diego/PycharmProjects/CasLAB-Predictive-Processing/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
RETOS_BEBRASK_dataset = pd.read_excel("RETOS_BEBRASK_long.xlsx")


In [3]:
rating_columns = [col for col in RETOS_BEBRASK_dataset.columns if 'Rating0' in col]
fulfilled_columns = [col for col in RETOS_BEBRASK_dataset.columns if 'Fulfilled' in col]
actions_dataset = RETOS_BEBRASK_dataset[rating_columns]
states_dataset = RETOS_BEBRASK_dataset[fulfilled_columns]
np.array(actions_dataset.iloc[0].values)
actions_dataset= actions_dataset-1

In [4]:
# Define the number of states and actions
num_states = 2
num_actions = 4

# Define the range for the reward values
Rmin, Rmax = -3, 3

# Initialize the rewards matrix with random values between Rmin and Rmax
rewards_rand = np.random.uniform(Rmin, Rmax, (num_states, num_actions))

print("Randomly initialized rewards matrix:")
print(rewards_rand)


Randomly initialized rewards matrix:
[[ 0.14134336  0.20208811  2.74055466  0.68072553]
 [ 1.11447484 -2.26380433 -0.46788622 -2.4489867 ]]


In [5]:
def calculate_rewards(states_df, actions_df,rewards):
    # Define the reward mappings for each state and action
    reward_mapping = {
        0: {0: rewards[0,0], 1: rewards[0,1], 2: rewards[0,2], 3: rewards[0,3]},
        1: {0: rewards[1,0], 1: rewards[1,1], 2: rewards[1,2], 3: rewards[1,3]}
    }
    
    # Initialize an empty DataFrame with the same shape as states_df
    
    rewards_df = pd.DataFrame(index=states_df.index, columns=range(1,46))
    
    # Iterate through each cell in the DataFrame
    for row in range(len(states_df.index)):
        for col in range(len(states_df.columns)):
            state = states_df.iloc[row, col]
            action = actions_df.iloc[row, col]
            # Set the reward using the mapping
            reward = reward_mapping[state][action]
            rewards_df.iloc[row, col] = reward
    
    return rewards_df


# Assuming states_dataset and actions_dataset are defined somewhere
# Example usage:
rewards_dataset = calculate_rewards(states_dataset, actions_dataset, rewards_rand)
rewards_dataset = rewards_dataset.apply(pd.to_numeric, errors='coerce')


In [6]:
def calculate_rewards_individual(states_df, actions_df,rewards):
    # Define the reward mappings for each state and action
    reward_mapping = {
        0: {0: rewards[0,0], 1: rewards[0,1], 2: rewards[0,2], 3: rewards[0,3]},
        1: {0: rewards[1,0], 1: rewards[1,1], 2: rewards[1,2], 3: rewards[1,3]}
    }
    
    rewards_list = []
    
    for state,action in zip(states_df,actions_df):
        rewards_list.append(reward_mapping[state][action])
    return rewards_list


In [7]:
test_subject_state = states_dataset.loc[0]
test_subject_actions = actions_dataset.loc[0]
test_subject_rewards = rewards_dataset.loc[0]

In [8]:
def perturb_rewards(rewards, scale=0.05):
    # Add small random changes to each reward, scaled by `scale`
    noise = np.random.randn(*rewards.shape) * scale
    new_rewards = rewards + noise # Assuming rewards are normalized [0,1]
    
    return new_rewards

In [32]:
for participant in states_dataset.index:
    print(participant)
    if participant == 0:
        continue
    
    
    best_loss = 10000
    best_init_rewards = rewards_rand
    states = torch.tensor(states_dataset.loc[participant].values, dtype=torch.int64)
    actions = torch.tensor(actions_dataset.loc[participant].values, dtype=torch.int64)

    for i in range(50):
        rewards_rand = np.random.uniform(Rmin, Rmax, (num_states, num_actions))
        rewards_dataset = calculate_rewards_individual(states_dataset.loc[participant], actions_dataset.loc[participant], rewards_rand)
        probabilities = torch.nn.functional.softmax(torch.tensor(rewards_rand[states]), dim=1)
        selected_probabilities = probabilities.gather(1, actions.unsqueeze(1)).squeeze(1)
        loss = -torch.sum(torch.log(selected_probabilities))
        if loss < best_loss:
            best_loss = loss
            best_init_rewards = rewards_rand
            rewards_opt = rewards_dataset
    
    best_rewards = best_init_rewards
    
    states = torch.tensor(states_dataset.loc[participant].values, dtype=torch.int64)
    actions = torch.tensor(actions_dataset.loc[participant].values, dtype=torch.int64)
    m = 0 
    while m<1000:
        print(m)
        beta_unconstrained_match = torch.tensor([-2.8], requires_grad=True)  # Initialize around 0 to start with sigmoid(beta) ~ 0.5
        beta_unconstrained_no_match = torch.tensor([-2.8], requires_grad=True)  # Initialize around 0 to start with sigmoid(beta) ~ 0.5

        
        optimizer = optim.Adam([beta_unconstrained_match,beta_unconstrained_no_match], lr=0.05)
        
        
        Q_values = torch.tensor(best_rewards, requires_grad=True)  # Enable gradients if directly influencing the loss
        rewards = torch.tensor(rewards_opt, dtype=torch.float32)

            # Training loop for current participant
        for epoch in range(250):
            optimizer.zero_grad()
            beta_match = torch.sigmoid(beta_unconstrained_match)   
            beta_no_match = torch.sigmoid(beta_unconstrained_no_match)   
    
            # Manual Q-value update simulation
            Q_updated = Q_values.clone()
            loss = 0
            probability_data = 1
            for i in range(1,len(states)):
                state = states[i]
                action = actions[i]
                reward = rewards[i-1]
                # Update Q_values on a new tensor to avoid in-place operations
                probabilities = torch.nn.functional.softmax(Q_updated, dim=1)  # Assuming Q_updated is indexed appropriately
                selected_probability = probabilities[state].gather(0, action)
                probability_data *= selected_probability
                loss -= torch.log(selected_probability)
                
                
                ## Maybe different beta for sequence s0->s0, s0->s1, s1->s0, s1->s1
                if (states[i-1] == 0):
                    beta = beta_no_match
                else:
                    beta = beta_match
                new_Q_value = beta * (Q_updated[state, action].clone()-reward) + Q_updated[state, action].clone()
                Q_updated[state, action] = new_Q_value

            # Compute loss based on the probabilities from Q_updated
    
            # Backward pass and optimization step
            loss.backward()
            optimizer.step()
            
        
        better = False
        scale = 0.02
        print("Loss:",loss)
        while(not better and m<1000):
            
            new_rewards = perturb_rewards(best_rewards,scale)
            rewards_opt = calculate_rewards_individual(states_dataset.loc[participant], actions_dataset.loc[participant], rewards_rand)
            
            Q_values = torch.tensor(new_rewards, requires_grad=True)  # Enable gradients if directly influencing the loss
            rewards = torch.tensor(rewards_opt, dtype=torch.float32)

            # Training loop for current participant
            Q_updated = Q_values.clone()
            new_probability_data = 1
            new_loss = 0
            for i in range(1,len(states)):
                
                
                state = states[i]
                action = actions[i]
                reward = rewards[i-1]
                # Update Q_values on a new tensor to avoid in-place operations
                probabilities = torch.nn.functional.softmax(Q_updated, dim=1)  # Assuming Q_updated is indexed appropriately
                selected_probability = probabilities[state].gather(0, action)
                new_probability_data *= selected_probability
                new_loss -= torch.log(selected_probability)

                
                ## Maybe different beta for sequence s0->s0, s0->s1, s1->s0, s1->s1
                if (states[i-1] == 0):
                    beta = beta_no_match
                else:
                    beta = beta_match
                new_Q_value = beta * (Q_updated[state, action].clone()-reward) + Q_updated[state, action].clone()
                Q_updated[state, action] = new_Q_value
            
            if new_probability_data/probability_data < 1:
                prob = new_probability_data/probability_data/2
            else:
                prob = new_probability_data/probability_data
            
            value_placeholder = min(1,prob)
            random_number = np.random.random()
            if random_number < value_placeholder:
                better = True
                best_rewards = new_rewards
            m+=1
    break
            

            
            


    
    
    
            
    

0
1
0
Loss: tensor(57.8049, dtype=torch.float64, grad_fn=<SubBackward0>)
1
Loss: tensor(57.8129, dtype=torch.float64, grad_fn=<SubBackward0>)
3
Loss: tensor(57.9254, dtype=torch.float64, grad_fn=<SubBackward0>)
4
Loss: tensor(57.9081, dtype=torch.float64, grad_fn=<SubBackward0>)
6
Loss: tensor(57.8528, dtype=torch.float64, grad_fn=<SubBackward0>)
7
Loss: tensor(57.6332, dtype=torch.float64, grad_fn=<SubBackward0>)
8
Loss: tensor(57.5705, dtype=torch.float64, grad_fn=<SubBackward0>)
9
Loss: tensor(57.5171, dtype=torch.float64, grad_fn=<SubBackward0>)
10
Loss: tensor(57.6476, dtype=torch.float64, grad_fn=<SubBackward0>)
11
Loss: tensor(57.8428, dtype=torch.float64, grad_fn=<SubBackward0>)
12
Loss: tensor(57.7767, dtype=torch.float64, grad_fn=<SubBackward0>)
13
Loss: tensor(57.6339, dtype=torch.float64, grad_fn=<SubBackward0>)
14
Loss: tensor(57.5564, dtype=torch.float64, grad_fn=<SubBackward0>)
15
Loss: tensor(57.6481, dtype=torch.float64, grad_fn=<SubBackward0>)
16
Loss: tensor(57.5111,

KeyboardInterrupt: 

In [34]:
print(f'Participant {participant}, Loss = {loss.item()}')
print("Optimized Q-values:")
print(Q_values)
print("Probabilities each Action")
print(torch.nn.functional.softmax(Q_updated,dim=1))
print("Optimized Beta Match:", beta_match.item())
print("Optimized Beta No Match:", beta_no_match.item())


Participant 1, Loss = 35.80282243844217
Optimized Q-values:
tensor([[ 1.9197,  1.8926,  0.8465, -0.8104],
        [-0.2910,  1.3701,  1.9152,  1.4996]], dtype=torch.float64,
       requires_grad=True)
Probabilities each Action
tensor([[0.4034, 0.4387, 0.1320, 0.0258],
        [0.0406, 0.2249, 0.4991, 0.2354]], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)
Optimized Beta Match: 0.015020608901977539
Optimized Beta No Match: 0.010009638033807278


In [11]:
print("Optimized Q-values:")
print(Q_values)
print("Probabilities each Action")
print(torch.nn.functional.softmax(Q_values,dim=1))


Optimized Q-values:
tensor([[ 2.1748, -0.0679, -0.1099,  1.3533],
        [-2.2362,  1.4799,  1.5897,  3.0284]], dtype=torch.float64,
       requires_grad=True)
Probabilities each Action
tensor([[0.6069, 0.0644, 0.0618, 0.2669],
        [0.0036, 0.1461, 0.1631, 0.6873]], dtype=torch.float64,
       grad_fn=<SoftmaxBackward0>)


In [104]:
  print(f'Participant {participant}, Loss = {new_loss.item()}')

Participant 0, Loss = 31.38513340512362
